<a href="https://colab.research.google.com/github/killerdds01/Pitone-IA/blob/main/progettoProgettoso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Cella 1: Importazioni e Configurazione Iniziale
# Questo blocco importa tutte le librerie necessarie e configura l'ambiente di esecuzione,
# inclusa la verifica della disponibilità di una GPU e l'impostazione del dispositivo.

# Importazioni PyTorch: Componenti essenziali per la costruzione e l'addestramento di reti neurali.
import torch
import torch.nn as nn                  # Moduli di rete neurale (es. layer, funzioni di attivazione)
import torch.optim as optim            # Algoritmi di ottimizzazione (es. SGD, Adam)
import torchvision                     # Libreria per computer vision (modelli, dataset, trasformazioni)
import torchvision.transforms as transforms # Trasformazioni di immagini per il preprocessing e data augmentation
import torchvision.datasets as datasets # Per caricare dataset comuni, inclusi ImageFolder
from torch.optim import lr_scheduler   # Scheduler per regolare il learning rate durante l'addestramento
from torch.utils.data import DataLoader, random_split # Per caricare dati in batch e dividere i dataset

# Importazioni per metriche e visualizzazioni: Strumenti per valutare il modello e visualizzare i risultati.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np                     # Per manipolare array numerici (fondamentale per PyTorch)
import matplotlib.pyplot as plt        # Per creare grafici e visualizzazioni
import seaborn as sns                  # Per grafici statistici più avanzati e esteticamente gradevoli

# Importazioni per utilità: Funzioni ausiliari per operazioni di sistema e manipolazione dati.
import time                            # Per misurare il tempo di esecuzione delle operazioni
import os                              # Per interagire con il sistema operativo (es. percorsi file, creazione directory)
import copy                            # Per creare copie profonde di oggetti (necessario per salvare i pesi del modello migliore)
import collections                     # Per contare le occorrenze degli elementi (utile per la distribuzione delle classi)

# Stampa le informazioni sull'ambiente PyTorch e GPU.
print(f"Pytorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# Imposta il dispositivo da utilizzare per l'addestramento: GPU (CUDA) se disponibile, altrimenti CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Pytorch Version: 2.6.0+cu124
CUDA Available: False
Using device: cpu


In [ ]:
# Cella 2: Configurazione Kaggle e Download/Decompressione Dataset
# Questo blocco gestisce l'autenticazione con Kaggle API, il download del dataset
# e la sua decompressione nella directory di lavoro di Colab.

# Carica il file 'kaggle.json' per l'autenticazione Kaggle API.
# Questo file contiene le tue credenziali API di Kaggle.
from google.colab import files
files.upload() # Si apre una finestra di dialogo per selezionare il file.

# Elenca i file nella directory corrente per verificare il caricamento.
!ls -l

# Configura Kaggle API: sposta il file kaggle.json nella directory corretta e imposta i permessi.
# Questo permette di utilizzare i comandi Kaggle CLI.
!mkdir -p ~/.kaggle # Crea la directory .kaggle se non esiste
!mv kaggle.json ~/.kaggle/kaggle.json # Sposta il file di credenziali
!chmod 600 ~/.kaggle/kaggle.json # Imposta i permessi per renderlo accessibile solo all'utente

# Scarica il Dataset "Multi class garbage classification Dataset" da Kaggle Hub.
# L'identificatore del dataset punta a una specifica risorsa su Kaggle.
import kagglehub
dataset_identifier = "vishallazrus/multi-class-garbage-classification-dataset"
path = kagglehub.dataset_download(dataset_identifier)
print("Path to new dataset files:", path)

# Decomprimi il dataset scaricato.
# Il file ZIP viene estratto in una directory specifica per l'organizzazione.
zip_file_name = "multi-class-garbage-classification-dataset.zip"
extraction_path = "./garbage_dataset_with_organic" # Cartella di destinazione per l'estrazione
!unzip -q {zip_file_name} -d {extraction_path} # Comando unzip in modalità 'quiet'
print(f"Dataset decompresso in: {extraction_path}")

# Esplora la sottocartella annidata del dataset per identificare il percorso corretto dei dati.
# Spesso i dataset Kaggle hanno una struttura di directory nidificata.
nested_path = "/kaggle/input/multi-class-garbage-classification-dataset/Multi class garbage classification"
print(f"\nContenuto di '{nested_path}':")
print(os.listdir(nested_path))

In [ ]:
# Cella 3: Caricamento e Preparazione del Dataset Rifiuti con PyTorch
# Questo blocco definisce le trasformazioni delle immagini, rimappa le classi
# del dataset originale alle classi target del progetto e crea i DataLoader
# per l'addestramento, la validazione e il test.

print("\n<<< Caricamento e Preparazione Dataset Rifiuti per PyTorch >>>")

# Definizione del percorso base del dataset dove si trovano le immagini.
DATA_PATH = '/kaggle/input/multi-class-garbage-classification-dataset/Multi class garbage classification'

# Mappatura delle classi originali del dataset alle 5 classi finali del progetto.
# Questo raggruppa 'paper' e 'cardboard' in 'carta' ed esclude 'metal'.
class_mapping_original_to_final = {
    'plastic': 0,      # Nuova classe 0: Plastica
    'paper': 1,        # Nuova classe 1: Carta
    'cardboard': 1,    # Mappa 'cardboard' a 'Carta'
    'glass': 2,        # Nuova classe 2: Vetro
    'compost': 3,      # Nuova classe 3: Organico
    'trash': 4         # Nuova classe 4: Indifferenziato
    # 'metal' NON è inclusa, verrà filtrata nella funzione di mappatura
}

# Definisce i nomi delle classi finali, utilizzati per etichettare i grafici e le predizioni.
final_class_names = ['plastica', 'carta', 'vetro', 'organico', 'indifferenziato']
num_classes = len(final_class_names) # Determina il numero totale di classi per il modello (5).

# Trasformazioni delle immagini: preprocessing e data augmentation.
# Queste trasformazioni sono applicate a ciascuna immagine prima di essere passata al modello.
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # Ritaglia casualmente e ridimensiona a 224x224 (data augmentation)
        transforms.RandomHorizontalFlip(), # Riflette orizzontalmente l'immagine (data augmentation)
        transforms.ToTensor(),             # Converte l'immagine PIL in un tensore PyTorch
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalizza l'immagine con media e deviazione standard di ImageNet
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),            # Ridimensiona l'immagine a 256x256
        transforms.CenterCrop(224),        # Esegue un ritaglio centrale a 224x224
        transforms.ToTensor(),             # Converte in tensore
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalizza
    ]),
}

# Caricamento dei dataset RAW utilizzando ImageFolder.
# Inizialmente, carica tutte le immagini e le loro classi originali senza trasformazioni.
train_dataset_original = datasets.ImageFolder(os.path.join(DATA_PATH, 'train'))
test_dataset_original = datasets.ImageFolder(os.path.join(DATA_PATH, 'test'))

# Funzione per filtrare e rimappare i campioni in base alla 'class_mapping_original_to_final'.
# Questo crea una lista di (percorso_immagine, nuova_etichetta_classe) solo per le classi incluse.
def get_remapped_samples(original_dataset, class_mapping):
    remapped_samples = []
    for path, original_class_idx in original_dataset.samples:
        original_class_name = original_dataset.classes[original_class_idx]
        if original_class_name in class_mapping:
            remapped_samples.append((path, class_mapping[original_class_name]))
    return remapped_samples

# Ottieni i campioni filtrati e rimappati per i set di addestramento e test.
train_remapped_samples = get_remapped_samples(train_dataset_original, class_mapping_original_to_final)
test_remapped_samples = get_remapped_samples(test_dataset_original, class_mapping_original_to_final)

# Crea un Custom Dataset che utilizza i campioni filtrati/rimappati e applica le trasformazioni.
class CustomRemappedDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
        self.classes = final_class_names # Associa i nomi delle classi finali al dataset
        self.class_to_idx = {name: i for i, name in enumerate(final_class_names)} # Mappa nomi a indici

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = datasets.folder.default_loader(path) # Carica l'immagine come PIL Image
        if self.transform:
            image = self.transform(image) # Applica le trasformazioni definite
        return image, label

# Inizializza i dataset finali con i campioni rimappati e le trasformazioni appropriate.
train_dataset_full = CustomRemappedDataset(train_remapped_samples, transform=data_transforms['train'])
test_dataset = CustomRemappedDataset(test_remapped_samples, transform=data_transforms['test'])

# Suddivide il dataset di addestramento completo in set di addestramento (80%) e validazione (20%).
train_size = int(0.8 * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size
train_dataset, val_dataset = random_split(train_dataset_full, [train_size, val_size])

# Creazione dei DataLoader: gestiscono il caricamento dei dati in batch e lo shuffling.
BATCH_SIZE = 64     # Numero di immagini per batch
num_workers = 2     # Numero di sottoprocessi per il caricamento dei dati (migliora le performance)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# Stampa le dimensioni dei set e il numero di batch per verifica.
print(f"Dimensioni del set di addestramento: {len(train_dataset)} immagini")
print(f"Dimensioni del set di validazione: {len(val_dataset)} immagini")
print(f"Dimensioni del set di test: {len(test_dataset)} immagini")
print(f"Numero di batch per l'addestramento: {len(train_loader)}")
print(f"Numero di batch per la validazione: {len(val_loader)}")
print(f"Numero di batch per il test: {len(test_loader)}")

# Controllo della distribuzione delle classi nei set di Training, Validazione e Test.
# Questo aiuta a verificare che la rimappatura e la suddivisione siano state corrette.
print("\nConvalida conteggio immagini per le NUOVE classi (Train/Val/Test):")
train_class_counts = collections.Counter([train_dataset_full.samples[idx][1] for idx in train_dataset.indices])
val_class_counts = collections.Counter([train_dataset_full.samples[idx][1] for idx in val_dataset.indices])
test_class_counts = collections.Counter([s[1] for s in test_dataset.samples])

print("Distribuzione classi Training (rimappate):")
for class_idx in sorted(train_class_counts.keys()):
    print(f"   {final_class_names[class_idx]}: {train_class_counts[class_idx]} immagini")

print("Distribuzione classi Validazione (rimappate):")
for class_idx in sorted(val_class_counts.keys()):
    print(f"   {final_class_names[class_idx]}: {val_class_counts[class_idx]} immagini")

print("Distribuzione classi Test (rimappate):")
for class_idx in sorted(test_class_counts.keys()):
    print(f"   {final_class_names[class_idx]}: {test_class_counts[class_idx]} immagini")


# --- Visualizzazione di Esempio di Immagini del Dataset ---
# Questo blocco mostra alcune immagini con le loro etichette rimappate.
class_names = final_class_names # Usa i nomi delle classi finali per la visualizzazione

# Funzione helper per denormalizzare e mostrare un'immagine tensore.
def imshow(img):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img = img * std + mean # Denormalizza l'immagine
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # Trasponi i canali per la visualizzazione Matplotlib
    plt.show()

# Ottieni un batch di immagini dal train_loader per la visualizzazione.
dataiter = iter(train_loader)
images, labels = next(dataiter)

plt.figure(figsize=(10, 5)) # Imposta la dimensione della figura
for i in range(4): # Mostra le prime 4 immagini del batch
    plt.subplot(1, 4, i + 1) # Crea un subplot per ciascuna immagine
    img = images[i]
    mean_val = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std_val = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img_denormalized = img * std_val + mean_val # Denormalizza per visualizzazione corretta
    npimg = img_denormalized.numpy()

    plt.imshow(np.transpose(npimg, (1, 2, 0))) # Mostra l'immagine
    plt.title(class_names[labels[i].item()]) # Imposta il titolo con il nome della classe
    plt.axis('off') # Nasconde gli assi
plt.tight_layout() # Ottimizza la disposizione dei subplot
plt.show() # Mostra la figura

# Stampa le forme dell'input e il numero di classi per la configurazione del modello.
input_shape = (3, 224, 224) # Formato (Canali, Altezza, Larghezza) per input del modello
print(f"Forma dell'input per il modello (Canali, Altezza, Larghezza): {input_shape}")
print(f"Numero totale di classi per il modello (DOPO raggruppamenti/esclusioni): {num_classes}")
print("\nDataset caricato e pronto per l'addestramento del modello!")

In [ ]:
# Cella 4: Architettura del Modello e Funzione di Addestramento
# Questo blocco definisce l'architettura del modello utilizzando il transfer learning con ResNet18,
# congela i layer pre-addestrati e definisce la funzione principale per l'addestramento
# e la valutazione del modello con Early Stopping.

print("\n<<< Definizione Architettura del Modello >>>")

# Carica il modello pre-addestrato ResNet18 da torchvision.
# 'weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1' scarica i pesi pre-addestrati su ImageNet.
model_transfer = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

# Sostituzione del layer finale completamente connesso (Fully Connected layer).
# Adattiamo l'ultimo layer della ResNet18 al nostro numero specifico di classi (5 classi di rifiuti).
num_ftrs = model_transfer.fc.in_features # Ottiene il numero di feature in input al layer FC originale
model_transfer.fc = nn.Linear(num_ftrs, num_classes) # Sostituisce il layer FC con uno nuovo per 'num_classes' uscite

# --- LOGICA DI CONGELAMENTO ROBUSTA ---
# Congela tutti i parametri del modello base tranne l'ultimo layer FC.
# Questo è il cuore del "fine-tuning leggero" o "feature extraction":
# solo i pesi del nuovo layer FC verranno addestrati, mentre il resto del modello rimane fisso.
for param in model_transfer.parameters():
    param.requires_grad = False # Imposta requires_grad a False per congelare il layer
for param in model_transfer.fc.parameters():
    param.requires_grad = True # Imposta requires_grad a True solo per il nuovo layer FC, rendendolo addestrabile
# --- FINE LOGICA DI CONGELAMENTO ROBUSTA ---

# Sposta il modello sul dispositivo di calcolo (GPU o CPU) definito in precedenza.
model_transfer = model_transfer.to(device)

# Stampa la struttura del modello modificato per verifica.
print("\nModello ResNet18 caricato e modificato:")
print(model_transfer)

# Verifica quali parametri del modello sono addestrabili.
print("\nParametri addestrabili (dovrebbe essere solo il layer fc):")
trainable_params_count = 0
for name, param in model_transfer.named_parameters():
    if param.requires_grad:
        print(name) # Stampa il nome del parametro se è addestrabile
        trainable_params_count += 1
if trainable_params_count == 0:
    print("ATTENZIONE: Nessun parametro addestrabile trovato! Questo causerà un errore di addestramento.")
elif trainable_params_count != 2: # Il layer FC ha solitamente 2 parametri: peso (weight) e bias
    print(f"ATTENZIONE: Trovati {trainable_params_count} parametri addestrabili. Ci si aspettano 2 (fc.weight, fc.bias).")

print("\nModello definito e pronto per l'addestramento!")


# --- Funzione di addestramento e valutazione (CON Early Stopping) ---
# Questa funzione incapsula l'intero ciclo di addestramento e validazione per più epoche.
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=10, phase_name="Fine-tuning"):
    since = time.time() # Inizia a misurare il tempo di addestramento

    best_model_wts = copy.deepcopy(model.state_dict()) # Inizializza con i pesi attuali del modello (per salvare i migliori)
    best_acc = 0.0 # Tiene traccia della migliore accuratezza di validazione trovata
    epochs_no_improve = 0 # Contatore delle epoche senza miglioramento per l'Early Stopping

    # Liste per memorizzare la loss e l'accuratezza per ogni epoca, per la visualizzazione dei grafici.
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Ogni epoca ha due fasi: 'train' (addestramento) e 'val' (validazione).
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # Imposta il modello in modalità addestramento (abilita dropout, batchnorm, ecc.)
                dataloader = train_loader # Usa il DataLoader del training set
            else:
                model.eval()  # Imposta il modello in modalità valutazione (disabilita dropout, fissa batchnorm)
                dataloader = val_loader # Usa il DataLoader del validation set

            running_loss = 0.0      # Inizializza la loss cumulativa per l'epoca
            running_corrects = 0    # Inizializza il conteggio delle predizioni corrette cumulative

            # Iterazione su tutti i batch del DataLoader per la fase corrente.
            for inputs, labels in dataloader:
                inputs = inputs.to(device) # Sposta input sul dispositivo (GPU/CPU)
                labels = labels.to(device) # Sposta etichette sul dispositivo

                optimizer.zero_grad() # Azzera i gradienti per evitare accumulazioni da batch precedenti

                # Calcolo del forward pass, loss e backward pass (solo in fase di addestramento).
                with torch.set_grad_enabled(phase == 'train'): # Abilita/Disabilita calcolo gradienti
                    outputs = model(inputs) # Esegue il forward pass
                    _, preds = torch.max(outputs, 1) # Ottiene le predizioni (indice della classe con prob. maggiore)
                    loss = criterion(outputs, labels) # Calcola la loss

                    if phase == 'train':
                        loss.backward() # Calcola i gradienti della loss rispetto ai parametri del modello
                        optimizer.step() # Aggiorna i pesi del modello usando l'ottimizzatore

                # Aggiorna le statistiche cumulative per l'epoca corrente.
                running_loss += loss.item() * inputs.size(0) # 'loss.item()' è la loss media per batch, la moltiplichiamo per la dimensione del batch
                running_corrects += torch.sum(preds == labels.data) # Conta le predizioni corrette

            # Calcola la loss e l'accuratezza medie per l'intera epoca.
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset) # .double() per divisione accurata

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Logica specifica per la fase di validazione.
            if phase == 'val':
                scheduler.step(epoch_loss) # Aggiorna lo scheduler del learning rate in base alla loss di validazione
                val_losses.append(epoch_loss) # Aggiunge la loss di validazione alla lista
                val_accuracies.append(epoch_acc.item()) # Aggiunge l'accuratezza di validazione alla lista

                # Logica di Early Stopping: Controlla se l'accuratezza di validazione è migliorata.
                if epoch_acc > best_acc: # Se l'accuratezza attuale è migliore della migliore finora
                    best_acc = epoch_acc # Aggiorna la migliore accuratezza
                    best_model_wts = copy.deepcopy(model.state_dict()) # Salva i pesi del modello corrispondenti
                    epochs_no_improve = 0 # Reset del contatore perché c'è stato un miglioramento
                else:
                    epochs_no_improve += 1 # Incrementa il contatore se non c'è stato miglioramento

            # Logica specifica per la fase di addestramento.
            else: # Corrisponde a 'phase == 'train''
                train_losses.append(epoch_loss) # Aggiunge la loss di addestramento
                train_accuracies.append(epoch_acc.item()) # Aggiunge l'accuratezza di addestramento

        # Controllo della condizione di Early Stopping dopo le fasi di train e val di ogni epoca.
        if epochs_no_improve >= patience: # Se il contatore supera la pazienza definita
            print(f"Early stopping triggerato: L'accuratezza di validazione non è migliorata per {patience} epoche.")
            break # Esce dal ciclo principale delle epoche (ferma l'addestramento)

        print() # Linea vuota per una migliore formattazione dell'output tra le epoche

    time_elapsed = time.time() - since # Calcola il tempo totale impiegato per l'addestramento
    print(f'Addestramento completato in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Migliore accuratezza di validazione: {best_acc:.4f}')

    model.load_state_dict(best_model_wts) # Carica i pesi del modello che ha dato la migliore accuratezza di validazione.
                                          # Questo garantisce che il modello restituito sia il "migliore" per la generalizzazione.

    # --- Plot delle curve di Loss e Accuratezza ---
    # Visualizza l'andamento della loss e dell'accuratezza nel tempo per monitorare l'addestramento.
    plt.figure(figsize=(12, 5)) # Imposta la dimensione della figura

    # Grafico della Loss
    plt.subplot(1, 2, 1) # Crea il primo subplot (1 riga, 2 colonne, primo grafico)
    plt.plot(train_losses, label='Train Loss') # Curva della loss di addestramento
    plt.plot(val_losses, label='Val Loss')     # Curva della loss di validazione
    plt.title(f'Curva di Loss ({phase_name})') # Titolo del grafico con il nome della fase
    plt.xlabel('Epoche') # Etichetta asse X
    plt.ylabel('Loss')   # Etichetta asse Y
    plt.legend()         # Mostra la legenda per identificare le curve
    plt.grid(True)       # Abilita la griglia

    # Grafico dell'Accuratezza
    plt.subplot(1, 2, 2) # Crea il secondo subplot
    plt.plot(train_accuracies, label='Train Accuracy') # Curva dell'accuratezza di addestramento
    plt.plot(val_accuracies, label='Val Accuracy')     # Curva dell'accuratezza di validazione
    plt.title(f'Curva di Accuratezza ({phase_name})') # Titolo del grafico
    plt.xlabel('Epoche')
    plt.ylabel('Accuratezza')
    plt.legend()
    plt.grid(True)

    plt.tight_layout() # Regola automaticamente i parametri del subplot per un layout compatto
    plt.show() # Mostra la figura con i grafici

    # Restituisce il modello addestrato e la storia delle metriche.
    return model, train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
# Cella 5: Esecuzione Addestramento (Fine-tuning Leggero) e Salvataggio Modello
# Questo blocco avvia la fase di addestramento "Fine-tuning Leggero" e, al termine,
# salva il modello con i pesi migliori (determinati dall'Early Stopping) nella sessione di Colab.

print("\n<<< Avvio Addestramento del Modello (Fase di Fine-tuning Leggero) >>>")

num_epochs_finetune = 60 # Numero massimo di epoche per questa fase

# --- Definizione della funzione di perdita (Loss Function) ---
criterion = nn.CrossEntropyLoss()

# --- Definizione dell'ottimizzatore ---
# Aggiorna SOLO i parametri che hanno requires_grad=True (che ora sarà SOLO il layer fc dalla Cella 4)
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_transfer.parameters()), lr=0.001)

# --- Definizione dello scheduler per il learning rate ---
scheduler_ft = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', patience=7, factor=0.1, min_lr=1e-08)

print(f"\n<<< Avvio Fine-tuning STANDARD per {num_epochs_finetune} epoche >>>")
print(f"Learning Rate iniziale: {optimizer_ft.param_groups[0]['lr']:.0e}")
print("Parametri addestrabili durante il Fine-tuning Standard (dovrebbe essere solo fc):")
trainable_params_count_check = 0
for name, param in model_transfer.named_parameters():
    if param.requires_grad:
        print(name)
        trainable_params_count_check += 1
if trainable_params_count_check == 0:
    print("ATTENZIONE: L'ottimizzatore non troverà parametri addestrabili. Controlla la Cella 3!")
elif trainable_params_count_check != 2: # fc.weight and fc.bias
    print(f"ATTENZIONE: Trovati {trainable_params_count_check} parametri addestrabili. Ci si aspettano 2 (fc.weight, fc.bias).")

# --- Esegui l'addestramento (SOLO Fine-tuning STANDARD - Fase 1) ---
# Pazienza per la prima fase: attendiamo 10 epoche senza miglioramento prima di fermarci
print("\n<<< Fase 1: Fine-tuning Leggero (Solo layer FC) >>>")
model_fine_tuned_light, train_loss_history_ft, val_loss_history_ft, train_acc_history_ft, val_acc_history_ft = train_model(
    model_transfer, criterion, optimizer_ft, scheduler_ft, num_epochs=num_epochs_finetune, patience=10, phase_name="Fine-tuning Leggero"
)

# --- Salva il modello con i pesi migliori della fase di fine-tuning leggero ---
# Questo sarà il modello con l'accuratezza di circa 82.45%
torch.save(model_fine_tuned_light.state_dict(), 'best_model_finetuned_light.pth')
print("\nModello 'best_model_finetuned_light.pth' salvato con successo nella sessione di Colab!")
print("\nFase 1 (Fine-tuning Leggero) completato!")

In [ ]:
# Cella 6: Valutazione Finale sul Test Set e Analisi Dettagliate
# Questo blocco carica il modello salvato e lo valuta sul set di test finale,
# fornendo metriche di performance come Loss, Accuratezza, Matrice di Confusione
# e un Report di Classificazione dettagliato.

print("\n<<< Avvio Valutazione Finale sul Test Set >>>")

# Carica il modello migliore salvato dalla fase di fine-tuning leggero.
# Assicurati che l'architettura del modello sia la stessa di quando è stato salvato.
# Per semplicità, riutilizziamo 'model_transfer' e gli carichiamo i pesi.
model_final_eval = model_transfer # Re-usa l'architettura già definita
model_final_eval.load_state_dict(torch.load('best_model_finetuned_light.pth'))
model_final_eval.to(device) # Assicurati che il modello sia sul dispositivo corretto
model_final_eval.eval() # Imposta il modello in modalità valutazione (disabilita dropout, batchnorm, ecc.)

print("Modello 'best_model_finetuned_light.pth' caricato con successo per la valutazione.")

# Esegui la valutazione sul test loader.
running_loss = 0.0
running_corrects = 0
all_labels = []
all_preds = []

with torch.no_grad(): # Disabilita il calcolo dei gradienti per la valutazione (risparmia memoria e tempo)
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_final_eval(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        all_labels.extend(labels.cpu().numpy()) # Raccogli tutte le etichette reali
        all_preds.extend(preds.cpu().numpy())   # Raccogli tutte le predizioni del modello

test_loss = running_loss / len(test_loader.dataset)
test_accuracy = running_corrects.double() / len(test_loader.dataset)

print("\nRisultati sul Test Set:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

print("\nValutazione sul Test Set completata!")

# --- Visualizzazione Matrice di Confusione e Report di Classificazione ---
# Questi strumenti forniscono una valutazione dettagliata delle performance del modello per ogni classe.

print("\n<<< Analisi delle Performance Dettagliate >>>")

# Calcola la Matrice di Confusione
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=final_class_names, yticklabels=final_class_names)
plt.title('Matrice di Confusione')
plt.xlabel('Classe Predetta')
plt.ylabel('Classe Reale')
plt.show()

# Genera il Report di Classificazione
# Questo report include precisione, richiamo, F1-score e supporto per ciascuna classe.
class_report = classification_report(all_labels, all_preds, target_names=final_class_names)
print("\nReport di Classificazione sul Test Set:")
print(class_report)

print("\nAnalisi completata!")